# 네이버뷰 크롤링 후 DB 적재

In [36]:
#모듈 import
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas  as pd 
import psycopg2
from datetime import date, timedelta
import re

In [37]:
# 자동 스크롤다운 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(5)

#select문 만들기
def select_command(table) :
    command = 'select * from ' + table
    return command

#insert문 만들기
def insert_command(table) :
    command = 'insert into ' + table + ' (title, date, url, hashtag) values (%s, %s, %s, %s);'
    return command

'뷔스티에 원피스', '롱패딩', '숏패딩', '롱코트', '숏코트', '잠옷', '홈웨어', '트위드자켓', '블레이저 자켓', '싱글 자켓', '크롭 자켓', '레더 자켓', '루즈핏', '테일러드자켓', '린넨자켓', '반팔자켓', '라이더 자켓', '오버핏 자켓', '청자켓', '니트가디건', '볼레로', '아가일', '크롭가디건', '반팔가디건', '앙고라', '바람막이', '아노락', '윈드브레이커', '바시티자켓', '바시티점퍼', '야구점퍼', '여름아우터', '간절기아우터', '겨울아우터', '니트 맨투맨', '기모 맨투맨', '크롭 맨투맨', '남자 맨투맨', '반집업', '반팔 맨투맨', '크롭후드', '기모후드', '후드티', '남자 후드', '꾸안꾸', '플리스', '후리스', '반팔티', '크롭 반팔', '블라우스', '와이셔츠', '셔츠', '체크셔츠', '타이다이', '옥스퍼드셔츠', '남방', '슬리브티', '니트', '남자니트', '조끼', '크롭티', '봄슬랙스', '여름슬랙스', '가을슬랙스', '겨울슬랙스', '간절기슬랙스', '데님팬츠', '연청', '진청', '중청', '흑청', '그레이진', '와이드팬츠', '밴딩바지', '스키니진', '린넨팬츠', '반바지', '하이웨스트', '로우웨스트', '부츠컷', '치노팬츠', '트레이닝복', '트레이닝바지', '조거팬츠', '카고바지', '일자팬츠', '냉장고바지', '멜빵바지', '오버롤팬츠', '찢청', '미니스커트', '미디스커트', '롱스커트', '플레어스커트', 'A라인 스커트', 'H라인 스커트', '청치마', '테니스스커트', '프릴스커트', '레더 스커트', '랩스커트', '데님스커트', '플리츠스커트', '미니원피스', '미디원피스', '롱원피스', '나시원피스', '코르셋', '셔츠원피스', '벨벳원피스', '꽃원피스', '골지원피스', '골지티', '쉬폰원피스', '쉬폰블라우스', '린넨원피스', '카라원피스', '랩원피스', '니트원피스', '후드원피스', '코듀로이', '반팔원피스', '점프슈트', '봄룩북', '봄옷쇼핑', '봄코디', '봄옷하울', '여름룩북', '여름옷쇼핑', '여름코디', '여름하울', '가을룩북', '가을옷쇼핑', '가을코디', '가을하울', '겨울룩북', '겨울옷쇼핑', '겨울코디', '겨울하울', '캠퍼스룩', '아메카지룩', '프레피룩', '하객룩', '놈코어룩', '시티보이룩', '미니멀룩', '애슬레저룩', '데이트룩', '출근룩', '데일리룩'

'침대', '매트리스', '토퍼', '소파', '테이블', '식탁', '책상', '거실장', 'tv장', '서랍', '수납장', '진열장', '책장', '선반', '행거', '옷장', '의자', '화장대', '콘솔','거울', '침구세트', '이불', '베개', '토퍼', '패드', '매트리스커버', '블라인드', '롤스크린', '커튼', '러그', '카페트', '매트', '쿠션', '방석', '소파커버', '조명', 'led평판등', '천장등', '장스탠드', '단스탠드', '데스크스탠드', '무드등', '장식조명', '벽조명', '센서등', '형광등', '집꾸미기', '방꾸미기', '인테리어'

In [62]:
keyword_list = []

In [63]:
driver = webdriver.Chrome('C:\chromedriver.exe')

for query_txt in keyword_list :

    #크롬드라이버 실행
    driver.get('http://www.naver.com')
    time.sleep(2)

    #키워드를 검색한 네이버창 접속
    element = driver.find_element_by_id("query")
    element.send_keys(query_txt)
    element.submit()

    # 뷰 클릭
    driver.find_element_by_link_text("VIEW").click()
    time.sleep(2)

    #필터 클릭
    driver.find_element_by_xpath('//*[@id="snb"]/div[1]/div/div[3]/a').click()
    time.sleep(2)

    #기간 1년 설정
    driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[8]').click()

    time.sleep(3)

    no2 = [ ]           # 게시글 번호 컬럼
    title2 = [ ]        # 게시물 제목 컬럼
    bdate2 = [ ]        # 작성 일자 컬럼
    url2 = [ ]          # url 컬럼
    hashtag2 = [ ]      # 해시태그 컬럼

    no = 1

    #더이상 스크롤이 안되면 종료
    i = 1
    time.sleep(1)

    #현재 스크롤 위치를 last_height로 받아옴
    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        #스크롤 한 번 실행 후 스크롤 위치를 new_height로 받아옴
        scroll_down(driver)
        new_height = driver.execute_script("return document.documentElement.scrollHeight")

        #더이상 스크롤이 안되면 종료
        if new_height == last_height:
            break

        last_height = new_height
        i += 1


    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    view_list = soup.find('ul','lst_total').find_all('li')
    print(query_txt, '포스트 개수 :', len(view_list))

    for i in view_list :
        
        no2.append(no)                            # 게시물 번호 리스트에 추가
        print(no, end=' ')

        all_title = i.find_all('a')
        title = all_title[5].get_text( )          # 게시물 제목
        title2.append(title)                      # 게시물 제목 리스트에 추가

        time.sleep(2)

        bdate = i.find('span','sub_time sub_txt').get_text( )  # 작성일자
        bdate.replace('.', '-')
        bdate2.append(bdate)                     # 작성일자 리스트에 추가

        time.sleep(2)

        url = str(all_title[0])
        url = url.split('"')[5]                 # 게시물 url
        url2.append(url)                        # url 리스트에 추가

        time.sleep(2)

        # 카페글은 hashtag가 없다면 댓글을 보여줌
        try :
            outline_test = i.find('div', 'total_tag_area').get_text()    
            
            # hashtag가 안달려있는 블로그 글도 있기때문에 오류 방지
            try:
                hashtag = " ".join(re.findall("#[가-힣]{1,}", outline_test))
                hashtag2.append(hashtag)
                    
            except Exception as e:
                hashtag = ""
                hashtag2.append(hashtag)

        except : 
            hashtag = ""
            hashtag2.append(hashtag)


        if no == 200 : #한번에 200개 까지만 크롤링 가능
            break
            
        no += 1

    print(query_txt, '끝')

    bdate3 = [] #전처리 후 작성일자

    # 작성일자를 날짜형식으로 통일화
    for day in bdate2 :

        if '시간 전' in day :       # n시간 전 업로드 글
            day = date.today()
        elif '분 전' in day :       # n분 전 업로드 글
            day = date.today()
        elif '일 전' in day :       # n일 전 업로드 글 
            day = date.today() - timedelta(int(day[0])) #8일전 글 부터는 날짜로 반환
        elif day == '어제' :        # 어제 업로드 글
            day = date.today() - timedelta(1)
        else :
            day = day.rstrip('.')
            day = day.replace('.', '-')
        
        bdate3.append(day)

    # df화
    naver_blog = pd.DataFrame()
    naver_blog['제목'] = title2
    naver_blog['작성일자'] = bdate3
    naver_blog['주소'] = url2
    naver_blog['해시태그'] = hashtag2

    #date-time 타입으로 변환
    naver_blog['작성일자'] = pd.to_datetime(naver_blog['작성일자'])

    print(naver_blog.tail())

    #postgresql 접속
    conn_string = "host='localhost' dbname='postgres' user='postgres' password='admin'"
    conn = psycopg2.connect(conn_string)
    cur = conn.cursor()

    table = 'view_interior'

    #table에 데이터 넣기
    for i in range(len(naver_blog)) :
        cur.execute(insert_command(table), (naver_blog.iloc[i][0], naver_blog.iloc[i][1], naver_blog.iloc[i][2], naver_blog.iloc[i][3]))
        conn.commit()

    #postgresql 연결종료
    conn.cursor().close()
    conn.close()

C:\Users\User\AppData\Local\Temp/ipykernel_8928/2412222373.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:\chromedriver.exe')
C:\Users\User\AppData\Local\Temp/ipykernel_8928/2412222373.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("query")
C:\Users\User\AppData\Local\Temp/ipykernel_8928/2412222373.py:15: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  driver.find_element_by_link_text("VIEW").click()
C:\Users\User\AppData\Local\Temp/ipykernel_8928/2412222373.py:19: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[1]/div/div[3]/a').click()
C:\Users\User\AppData\Local\Temp/ipykernel_8928/2412222373.py:23: DeprecationWarnin

조명 포스트 개수 : 82
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 조명 끝
                                  제목       작성일자  \
77                       간접조명 질문드립니다 2022-05-03   
78            식탁 조명등 루이스 로즈골드 골라봤어요. 2022-05-09   
79                 간접조명 무슨색으로 하셨나요?? 2022-05-30   
80  광명조명 광명역써밋플레이스 거실과 주방 천장 조명시공 후기 2022-05-04   
81                                조명 2022-05-27   

                                              주소 해시태그  
77        https://cafe.naver.com/kimyoooo/892283       
78        https://cafe.naver.com/overseer/769072       
79        https://cafe.naver.com/overseer/779847       
80        https://cafe.naver.com/overseer/766953       
81  https://cafe.naver.com/jihosoccer123/1469698       


C:\Users\User\AppData\Local\Temp/ipykernel_8928/2412222373.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("query")
C:\Users\User\AppData\Local\Temp/ipykernel_8928/2412222373.py:15: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  driver.find_element_by_link_text("VIEW").click()
C:\Users\User\AppData\Local\Temp/ipykernel_8928/2412222373.py:19: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[1]/div/div[3]/a').click()
C:\Users\User\AppData\Local\Temp/ipykernel_8928/2412222373.py:23: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[8]').click()


led평판등 포스트 개수 : 89
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 led평판등 끝
                                   제목       작성일자  \
84         DIY용 LED 엣지평판등 본체 새제품 3개일괄 2022-01-18   
85         이홈에서 구매한 led 엣지평판등 재판매합니다. 2021-06-04   
86                      LED평판등 조명 12개 2022-05-28   
87  씨티전기(주) LED평판등600*600/ 15개일괄판매합니다 2022-05-31   
88                 LED 평판등 판매합니다(60개) 2021-09-30   

                                              주소 해시태그  
84          https://cafe.naver.com/hssoon/306961       
85          https://cafe.naver.com/hssoon/283284       
86  https://cafe.naver.com/joonggonara/919788775  #조명  
87  https://cafe.naver.com/joonggonara/920232566       
88        https://cafe.naver.com/wjsekdtk/123640       


C:\Users\User\AppData\Local\Temp/ipykernel_8928/2412222373.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("query")
C:\Users\User\AppData\Local\Temp/ipykernel_8928/2412222373.py:15: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  driver.find_element_by_link_text("VIEW").click()
C:\Users\User\AppData\Local\Temp/ipykernel_8928/2412222373.py:19: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[1]/div/div[3]/a').click()
C:\Users\User\AppData\Local\Temp/ipykernel_8928/2412222373.py:23: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[8]').click()


...

집꾸미기 포스트 개수 : 89
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 집꾸미기 끝
                        제목       작성일자  \
84               보리집 꾸미기 끝 2021-10-21   
85     독립하고 처음 하는 요즘 집 꾸미기 2022-02-20   
86                햄스터 집꾸미기 2021-07-06   
87  얼른 신혼집 입주해서 집꾸미기 시작하기! 2022-04-13   
88          나름 집꾸미기 중입니다.. 2022-05-22   

                                             주소 해시태그  
84  https://cafe.naver.com/myfriendspet/1527130       
85  https://cafe.naver.com/culturebloom/1504353       
86  https://cafe.naver.com/myfriendspet/1454654       
87     https://cafe.naver.com/wedding055/796692       
88        https://cafe.naver.com/ghdi58/5138980       


C:\Users\User\AppData\Local\Temp/ipykernel_8928/2412222373.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("query")
C:\Users\User\AppData\Local\Temp/ipykernel_8928/2412222373.py:15: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  driver.find_element_by_link_text("VIEW").click()
C:\Users\User\AppData\Local\Temp/ipykernel_8928/2412222373.py:19: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[1]/div/div[3]/a').click()
C:\Users\User\AppData\Local\Temp/ipykernel_8928/2412222373.py:23: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[8]').click()


방꾸미기 포스트 개수 : 99
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 방꾸미기 끝
                  제목       작성일자                                         주소  \
94  아이방꾸미기 안전하고 편하고~ 2022-03-03       https://cafe.naver.com/baby8/2291959   
95    7세 여아방 공주방 꾸미기 2022-03-24  https://cafe.naver.com/kidsinlove/1015230   
96         아기방 꾸미기ㅎㅎ 2022-05-25  https://cafe.naver.com/momroom2013/634876   
97    레린이의 가성비 방꾸미기! 2022-04-12       https://cafe.naver.com/fx8300/921351   
98         방꾸미기 스티커? 2022-03-13  https://cafe.naver.com/ipocheonmom/447013   

       해시태그  
94  #아이방꾸미기  
95           
96           
97           
98           


C:\Users\User\AppData\Local\Temp/ipykernel_8928/2412222373.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("query")
C:\Users\User\AppData\Local\Temp/ipykernel_8928/2412222373.py:15: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  driver.find_element_by_link_text("VIEW").click()
C:\Users\User\AppData\Local\Temp/ipykernel_8928/2412222373.py:19: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[1]/div/div[3]/a').click()
C:\Users\User\AppData\Local\Temp/ipykernel_8928/2412222373.py:23: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[8]').click()


인테리어 포스트 개수 : 66
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 인테리어 끝
                           제목       작성일자  \
61                   25평 인테리어 2022-05-10   
62  10평 작은 술집 반셀프 인테리어 해보신 분? 2022-05-19   
63                  인테리어 견적문의 2022-05-27   
64                    인테리어 필름 2022-05-09   
65           샷시 인테리어 필름 시공 후기 2022-05-18   

                                              주소 해시태그  
61       https://cafe.naver.com/ilovegm1/2229283       
62  https://cafe.naver.com/jihosoccer123/1452659       
63        https://cafe.naver.com/overseer/779061       
64        https://cafe.naver.com/overseer/769117       
65        https://cafe.naver.com/overseer/774181       


#postgresql 접속
conn_string = "host='localhost' dbname='postgres' user='postgres' password='admin'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

In [ ]:
#table 확인하기
cur.execute(select_command(table))
conn.commit()
cur.fetchall()

In [17]:
#db 연결 종료
conn.cursor().close()
conn.close()

In [52]:
#varchar(500) type에 안들어가면 실행
b = 0

for i in range(len(title2)) :
    a = len(title2[i])

    if a > b :
        b = a
        print(i, b)

print(title2[20])

0 54
4 62
20 107
< 박강수 화보집 > ( 코드 데이브 Dac + M 스케일러로 들어보기 제3부 9/15회. 하울의 움직이는 성 9회 ) : 박강수 ~ 가을은 참 예쁘다 ( 2009. 3 ) & 기억날 그...
